In [1]:
import os
import torch
os.environ["https_proxy"] = "http://192.168.5.68:38080"
# os.environ["https_proxy"] =""
os.environ["http_proxy"] = ""

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='2'

# print(torch.cuda.current_device)
# torch.cuda.set_device(0)

## step1 导入包

In [2]:


from transformers import AutoModel,AutoModelForSequenceClassification,BertForSequenceClassification
from datasets import load_dataset,Dataset
import pandas as pd




## Step2 读取数据

In [3]:
pd_data =pd.read_excel("./data/emotion_tag_hjy_0506.xls")
pd_data.shape

(1400, 3)

## Step3 数据处理转换

In [3]:
tag_to_label = {
        "悲伤": 0,
        "愤怒": 1,
        "惊讶": 2,
        "恐惧": 3,
        "快乐": 4,
        "其他": 5,
        "厌恶": 6
    }

label_to_tag = {
         0:"悲伤",
         1:"愤怒",
         2:"惊讶",
         3:"恐惧",
         4:"快乐",
         5:"其他",
         6:"厌恶"
    }
pd_data['label']=pd_data["tag"].map(lambda x: tag_to_label[x])
pd_data['content_cn']=pd_data["content_cn"].map(lambda c: c.strip())



NameError: name 'pd_data' is not defined

In [5]:
dataset = Dataset.from_pandas(pd_data)
dataset = dataset.shuffle(seed=42)
dataset = dataset.filter(lambda x: x["content_cn"] is not None)
dataset = dataset.filter(lambda x: x["label"] is not None)

Filter:   0%|          | 0/1400 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [6]:
dataset = dataset.train_test_split(test_size =0.1)

dataset

DatasetDict({
    train: Dataset({
        features: ['ir_sid', 'content_cn', 'tag', 'label'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['ir_sid', 'content_cn', 'tag', 'label'],
        num_rows: 140
    })
})

## Step4 模型加载

In [7]:

from transformers import AutoTokenizer

mode_name="hfl/chinese-macbert-large"
# mode_name="google-bert/bert-base-chinese"
# model = AutoModel.from_pretrained(mode_name)
model = AutoModelForSequenceClassification.from_pretrained(mode_name,num_labels=len(label_to_tag),
        label2id=tag_to_label,
        # problem_type = "single_label_classification",
        id2label=label_to_tag)
tokenizer = AutoTokenizer.from_pretrained(mode_name)
tokenizer
# model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-macbert-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertTokenizerFast(name_or_path='hfl/chinese-macbert-large', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

## Step5 tokenizer 处理数据

In [8]:


def data_procce_function(example):
    token_exmaple = tokenizer(example['content_cn'],max_length=512,truncation=True)
    token_exmaple['labels'] = example['label']
    return token_exmaple


token_dataset = dataset.map(function=data_procce_function,batched=True,remove_columns=dataset["train"].column_names)
# tokenizer
token_dataset
# token_dataset["train"][0]
tokenizer.decode(token_dataset["train"][0]['input_ids'])


Map:   0%|          | 0/1260 [00:00<?, ? examples/s]

Map:   0%|          | 0/140 [00:00<?, ? examples/s]

'[CLS] 已 经 执 政 四 分 之 一 世 纪 的 普 京 再 度 以 压 倒 性 优 势 赢 得 选 举 。 虽 然 西 方 指 责 反 对 派 缺 席 的 这 场 选 举 不 公 平 ， 但 如 此 高 的 得 票 率 还 是 表 明 了 民 众 对 普 京 的 支 持 和 认 可 。 为 什 么 有 这 个 结 果 ？ 主 要 有 两 个 原 因 。 一 ， 是 民 众 对 叶 利 钦 时 代 的 苦 难 依 然 记 忆 犹 新 。 叶 利 钦 执 政 八 年 ， 按 西 方 的 标 准 是 最 [UNK] 民 主 [UNK] 的 时 期 ， 也 是 与 西 方 关 系 最 好 的 时 期 。 但 就 是 这 个 时 期 带 给 俄 罗 斯 无 穷 无 尽 的 灾 难 ： 经 济 崩 溃 、 人 均 gdp 下 降 50 % 以 上 、 通 货 膨 胀 超 过 2000 % ， 民 众 多 年 的 积 蓄 一 夜 之 间 化 为 乌 有 、 土 地 广 袤 的 国 家 70 % 的 食 品 竟 然 依 靠 进 口 。 在 这 样 的 状 况 下 ， 苏 联 时 期 建 立 的 社 会 保 险 、 免 费 医 疗 等 也 形 同 虚 设 。 根 据 联 合 国 粮 农 组 织 标 准 ， 恩 格 尔 系 数 50 % - 60 % 为 温 饱 ； 30 % - 40 % 为 相 对 富 裕 。 在 1990 年 以 前 ， 俄 罗 斯 恩 格 尔 系 数 为 30 % ， 而 1995 年 为 53 % ， 整 个 国 家 从 相 对 富 裕 降 为 温 饱 。 至 于 连 养 老 金 都 领 不 到 的 民 众 ， 温 饱 都 是 奢 求 。 西 方 民 主 之 于 俄 猛 于 病 毒 于 是 在 俄 罗 斯 ， 出 现 各 种 难 以 想 像 的 情 景 ： 前 往 车 臣 作 战 的 军 人 连 基 本 生 活 费 都 发 不 出 ； 民 众 买 烟 居 然 是 一 根 一 根 的 买 ； 俄 罗 斯 出 现 了 人 类 历 史 上 罕 见 的 和 平 时 期 人 均 寿 命 大 幅 下 降 的 现 象 （ 下 降 3. 5 岁 ） ， 尤 其 是 男 性 竟 然 为 57 岁 ， 低 于 领 退 休 金 的 年 龄 。 幅 度 超 过 美 国 混 

## Step6 创建评估函数

In [9]:
import evaluate,numpy 

acc_metric = evaluate.load("accuracy")
# f1_mectric = evaluate.load("f1")
def eval_metric(eval_predict):
    predictions,labels = eval_predict
    # print(predictions)
    # predictions = predictions.argmax(axis=-1)
    predictions = numpy.argmax(predictions, axis=-1)
    result = acc_metric.compute(predictions=predictions, references=labels)
    # if len(result) > 1:
    #     result["combined_score"] = numpy.mean(list(result.values())).item()
    return result
    


## Step7 创建训练参数

In [10]:
from transformers import TrainingArguments



train_args = TrainingArguments(
                               
                               output_dir="./output",
                               # per_device_eval_batch_size=64,
                               # per_device_train_batch_size=4,
                               logging_steps=50,
                               # logging_dir='./logs',
                               logging_strategy='steps',
                               # logging_first_step=True,
                               # save_strategy="epoch",
                               # use_mps_device="2",
                               num_train_epochs=5,  
                               # learning_rate=2e-5,
                               # weight_decay=0.01,
                               evaluation_strategy="steps",
                               metric_for_best_model="accuracy",
                               load_best_model_at_end=True

                               # per_device_train_batch_size=1,   # 训练时的batch_size
                               # gradient_accumulation_steps=32,  # *** 梯度累加 ***
                               # gradient_checkpointing=True,     # *** 梯度检查点 ***
                               # optim="adafactor",               # *** adafactor优化器 *** 
                               # per_device_eval_batch_size=1,    # 验证时的batch_size
                               # num_train_epochs=1,              # 训练轮数
                               # logging_steps=10,                # log 打印的频率
                               # evaluation_strategy="epoch",     # 评估策略
                               # save_strategy="epoch",           # 保存策略
                               # save_total_limit=3,              # 最大保存数
                               # learning_rate=2e-5,              # 学习率
                               # weight_decay=0.01,               # weight_decay
                               # metric_for_best_model="f1",      # 设定评估指标
                               # load_best_model_at_end=True
                              )
# train_args

In [11]:
from transformers import Trainer,DataCollatorWithPadding
import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# torch.backends.cudnn.enable =True
# torch.backends.cudnn.benchmark = True

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(model=model,
               args=train_args,
               train_dataset=token_dataset['train'],
               eval_dataset=token_dataset['test'],
               data_collator=data_collator,
               compute_metrics=eval_metric
                )
train_result = trainer.train()

# for obj in trainer.state.log_history:
#     logging.info(str(obj))

Step,Training Loss,Validation Loss,Accuracy
50,1.947100,1.831960,0.278571
100,1.581200,1.484488,0.464286
150,1.364500,1.341561,0.500000
200,1.155600,1.130627,0.600000
250,1.025100,1.032243,0.685714
300,1.037900,0.976122,0.707143
350,0.717800,1.057946,0.671429
400,0.556900,1.180457,0.678571
450,0.600900,1.072801,0.707143
500,0.334400,1.474848,0.657143


## Step8模型评估


In [12]:
metrics = train_result.metrics
trainer.log_metrics("train", metrics)

***** train metrics *****
  epoch                    =        5.0
  total_flos               =  5468043GF
  train_loss               =     0.7113
  train_runtime            = 0:08:52.56
  train_samples_per_second =     11.829
  train_steps_per_second   =      1.483


## Step 9 模型预测

In [13]:
from transformers import pipeline
#input='4月10日｜阿里巴巴(9988.HK)今日表现强势，盘中最高涨5.82%报74.6港元，股价创3月14日以来新高。消息面上，阿里巴巴创始人马云于阿里内网发帖，高度肯定蔡崇信和吴泳铭组成的新管理层的变革勇气，称阿里巴巴已重回健康成长轨道，并支持继续改革。这是退休后的马云五年来首次长篇幅分享对公司改革创新及未来前景的思考。'
#input='这位辛迪·坎宁安明星说，当被告知她在28年后被解雇时，她“心碎”了......阅读整个故事'
#input='阿尔贝松杂货店似乎发现自己成为了更多抖音诽谤的接收者，但这一次，该应用程序的一些人认为DoorDash可能与一名用户的碎牛肉剧有关。Shantaq(@heyshantaqtv)上传了一段病毒视频，呼吁阿尔贝松和...阅读整个故事'
input='洪水桥房协地盘四级火，起火逾18小时仍然未救熄。　　消防早上继续灌救，不断向洪雅路地盘火场射水，地盘仍然冒起浓烟，出动无人机升空视察火场情况。　　两个天秤一度被指有倒塌风险，屋宇署评估后沒有即时危险。　　有附近居民指有浓烟涌入家中，需关窗和戴上口罩。洪福邨居民何先生：「烟味很臭，我住在2楼也嗅到烟味，很像胶味、刺喉。今天好一些、昨天（星期二）厉害。昨天我不敢上去。」洪福邨居民黄小姐：「嗅到、仍很大烟味。担心味道、始终它烧了很久，烟也真的大。（家中有甚么应对措施？）把窗全都关掉、开冷气'

pipe =pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
pipe(input)

[{'label': '恐惧', 'score': 0.9721205234527588}]